# ML Pipeline Preparation

### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
import nltk
nltk.download(['punkt','wordnet','stopwords'])

import pandas as pd
from sqlalchemy import create_engine
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import pickle

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mathijn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mathijn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mathijn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Load data from database

In [2]:

engine = create_engine('sqlite:///..\data\DisasterResponse.db')
df = pd.read_sql_table('messages', engine) 
X = df.message
Y = df.iloc[:,4:]

### 2. Tokenization function to process the text data

In [3]:
def tokenize(message):    

    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+' 
          
    # Removing any URLs
    detected_urls = re.findall(url_regex, message)

    for url in detected_urls:
        message = message.replace(url, "urlplaceholder")

    # Removing punctuation
    message = re.sub(r"[^a-zA-Z0-9]", " ", message) 

    # Tokenizing the message
    tokens = word_tokenize(message)           

    # Lemmatize, lowercase, strip and also removing stopwords    
    clean_tokens = [WordNetLemmatizer().lemmatize(t).lower().strip() for t in tokens if t not in stopwords.words("english")]

    return(clean_tokens)
    

### 3. Build a machine learning pipeline
This machine pipeline takea in the `message` column as input and output classification results on the other 36 categories in the dataset. The countvectorizer calls the tokenizer function defined above. The transformed data is passed to the TfidTransformer and finally the data is fitted with a random forest classifier. The multi output classifier is used because the categories are not mutually exlusive: more labels can exist for each message.

In [4]:
pipeline = Pipeline([

    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())) 

])

First, a randomized search of a parameter grid in order to keep computation time down. 

In [8]:
def random_cv(pipeline):
    
    parameters = {   
        'vect__ngram_range': [(1,1),(1,2),(2,2)],     
        'vect__max_df': (0.25, 0.5, 0.75, 1),
        'vect__max_features': (None, 2000, 4000, 6000, 8000, 10000),
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [10, 40, 70, 100, 130, 160, 190, 230],
        'clf__estimator__max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
        'clf__estimator__max_features': ['auto', 'sqrt'], 
        'clf__estimator__min_samples_leaf': [1, 2, 4],
        'clf__estimator__min_samples_split': [2, 5, 10],
        'clf__estimator__bootstrap': [True, False]
    }

    cv = RandomizedSearchCV(pipeline, 
            param_distributions = parameters,
            scoring = ['f1_micro','precision_micro','recall_micro'],
            refit ='f1_micro',
            verbose = 3,
            n_jobs = -1,
            n_iter = 80      
            )

    return(cv)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
    test_size=0.2, 
    random_state=42)

model = random_cv(pipeline)
model.fit(X_train, y_train) 
model.best_params_

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 19.2min
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed: 94.8min
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed: 245.8min
[Parallel(n_jobs=-1)]: Done 400 out of 400 | elapsed: 353.5min finished


{'vect__ngram_range': (1, 1),
 'vect__max_features': 2000,
 'vect__max_df': 0.75,
 'tfidf__use_idf': True,
 'clf__estimator__n_estimators': 160,
 'clf__estimator__min_samples_split': 10,
 'clf__estimator__min_samples_leaf': 1,
 'clf__estimator__max_features': 'sqrt',
 'clf__estimator__max_depth': 80,
 'clf__estimator__bootstrap': True}

### 5. Test model
The precision, recall, and F1 score of the best model of the random search are shown for each of the labels. Also, the best fitting parameters are printed.

In [10]:
y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred,target_names=Y.columns))

precision    recall  f1-score   support

               related       0.82      0.96      0.89      3998
               request       0.84      0.49      0.62       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.76      0.68      0.72      2164
          medical_help       0.68      0.14      0.23       435
      medical_products       0.74      0.14      0.24       279
     search_and_rescue       0.55      0.08      0.14       136
              security       0.00      0.00      0.00        96
              military       0.59      0.11      0.18       158
                 water       0.86      0.52      0.65       335
                  food       0.85      0.75      0.80       584
               shelter       0.80      0.52      0.63       468
              clothing       0.76      0.19      0.30        70
                 money       0.77      0.09      0.16       112
        missing_people       1.00      0.02      0.03        63

### 6. Improve model
Use grid search to find better parameters. Here we are finetuning the best parameters found by the randomsearch to see if we can improve model fit further.

In [11]:
def grid_cv(pipeline):
    
    parameters = {   
        'vect__ngram_range': [(1,1)],     
        'vect__max_df': (0.7, 0.75, 0.8),
        'vect__max_features': (1000, 2000, 3000),
        'clf__estimator__n_estimators': [160],
        'clf__estimator__max_features': ["sqrt"], 
        'clf__estimator__min_samples_split': [5] 
    }

    cv = GridSearchCV(pipeline,
        param_grid = parameters,
        scoring = ['f1_micro','precision_micro','recall_micro'],
        refit ='f1_micro',
        verbose = 3,
        n_jobs = 4                  
        )

    return(cv)


In [12]:
model = grid_cv(pipeline)
model.fit(X_train, y_train) 
model.best_params_

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed: 52.5min
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed: 103.3min finished


{'clf__estimator__max_features': 'sqrt',
 'clf__estimator__min_samples_split': 5,
 'clf__estimator__n_estimators': 160,
 'vect__max_df': 0.7,
 'vect__max_features': 2000,
 'vect__ngram_range': (1, 1)}

### 7. Test model
Show the accuracy, precision, and recall of the tuned model. The improvements are marginal over the best model from the randomsearch. 

In [13]:
y_pred = model.predict(X_test)

print(classification_report(y_test,y_pred,target_names=Y.columns))

precision    recall  f1-score   support

               related       0.85      0.93      0.89      3998
               request       0.82      0.50      0.62       891
                 offer       0.00      0.00      0.00        24
           aid_related       0.74      0.71      0.73      2164
          medical_help       0.67      0.16      0.26       435
      medical_products       0.68      0.16      0.26       279
     search_and_rescue       0.56      0.10      0.17       136
              security       0.50      0.01      0.02        96
              military       0.59      0.11      0.18       158
                 water       0.83      0.56      0.66       335
                  food       0.84      0.76      0.80       584
               shelter       0.80      0.53      0.64       468
              clothing       0.88      0.21      0.34        70
                 money       0.69      0.08      0.14       112
        missing_people       1.00      0.03      0.06        63

### 9. Export model as a pickle file

In [24]:
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)